In [12]:
from nexusformat.nexus import *

## Loading an existing NeXuS sample file

In [13]:
a=nxload('data/SANSLOQCan2D.nxs')

In [ ]:
print(a.tree)

## Minimal new NeXuS file

In [15]:
import numpy as np

x=y=np.linspace(0,2*np.pi,101)
X,Y=np.meshgrid(x,y)
z=np.sin(X)*np.sin(Y)
a=NXdata(z,[y,x])
# a.save('function.nxs')

In [ ]:
b=nxload('function.nxs')
print(b.tree)

## Converting SANS-2 data to NeXuS

In [ ]:
# List of parameters to extract from current file and input into the new structure
# TODO: load these from original file, perhaps add a conversion function to existing class

detector_data = np.random.random_integers(0, 100, (1024,1024))
wavelength = 6.0 # AA
wavelength_spread = 0.1 # delta_L/L0
detector_sample_dist = 4 # m
w = 0.5 # m
h = 0.5 # m
N_pixel_x = 1024
N_pixel_y = 1024
x_pixel_size = w/N_pixel_x
y_pixel_size = h/N_pixel_y

In [ ]:
from sansdata import *
sample = SansData('data/memb_BS_Q1_6_0Ang.mpa')

In [23]:
detector_data = sample.raw_intensity
wavelength = round(sample.L0,4)
wavelength_spread = 0.1
detector_sample_dist = sample.d
# Original code seems to have a different pixel size on each axis
# Verify that this is correct and where this is coming from
pixel_size = 0.5 / 1024
x_pixel_size = pixel_size
y_pixel_size = pixel_size

In [ ]:
# Should conform to a minimal version of the NXsas application definition as specified in
# https://manual.nexusformat.org/classes/applications/NXsas.html

source = NXsource(type = 'Reactor Neutron Source', probe = 'neutron')
monochromator = NXmonochromator(wavelength=NXfield(wavelength, units='angstrom'), wavelength_spread=wavelength_spread)
detector = NXdetector(data=detector_data, distance=NXfield(detector_sample_dist, units='m'), x_pixel_size=NXfield(x_pixel_size, units='m'), y_pixel_size=NXfield(y_pixel_size, units='m'))
instr = NXinstrument(name = 'SANS-2', SOURCE=source, MONOCHROMATOR=monochromator, DETECTOR=detector)
# TODO: add collimator 
data = NXdata()
data['@signal'] = 'data'
entry = NXentry(definition = 'NXsas', INSTRUMENT=instr, DATA=data)
entry['DATA/data']=NXlink(entry['INSTRUMENT/DETECTOR/data'])
print(entry.tree)

In [ ]:
entry.save('sans-2-RID.nxs', mode='w+')

In [ ]:
b=nxload('sans-2-RID.nxs')
print(b.tree)